# Lesson 2: RAG Triad of metrics

In [33]:
!pip install llama-index-core llama-index-utils-workflow  --quiet
!pip install llama-index-llms-openai-like --quiet
!pip install llama-index-embeddings-huggingface --quiet
!pip install llama-index --quiet
!pip install trulens trulens-eval trulens-providers-openai trulens-providers-litellm --quiet
!pip install trulens-apps-llamaindex --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [34]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
import os
import getpass

INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"

from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(
  model=MODEL_NAME,
  api_key=API_KEY,
  api_base= f"{INFERENCE_SERVER_URL}/v1",
  context_window=1234,
  is_chat_model=True,  # supports chat completions
  is_function_calling_model=True # supports tools/functions in the api
)


In [36]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [37]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.postprocessor import SentenceTransformerRerank

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [38]:
from llama_index.core import Document

document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

In [39]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
text_splitter = SentenceSplitter()


from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = HuggingFaceEmbedding()

Settings.llm = llm
Settings.embed_model = embed_model
# Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.node_parser = node_parser
Settings.text_splitter = text_splitter
Settings.num_output = 512
Settings.context_window = 4096

In [40]:
#Node extraction
nodes = node_parser.get_nodes_from_documents(documents)
base_nodes = text_splitter.get_nodes_from_documents(documents)

#Index building
sentence_index = VectorStoreIndex(nodes)
base_index = VectorStoreIndex(base_nodes)

In [41]:
postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

query_engine = sentence_index.as_query_engine(
    similarity_top_k=6,
    # the target key defaults to `window` to match the node_parser's default
     node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

window_response = query_engine.query(
    "How do you create your AI portfolio?"
)
print(window_response)

To create an AI portfolio, focus on showcasing a progression of projects from simple to complex over time. This demonstrates your growth in skills. Start with personal projects, such as mimicking the sin(x) function with a neural network, as a foundational learning experience. Progress to more intricate projects, like applying machine learning to various industries like manufacturing, healthcare, climate change, agriculture, ecommerce, and advertising. Ensure each project highlights your ability to identify valuable ideas and apply them effectively. Additionally, consider collaborative projects to showcase your leadership skills, even without formal management roles. Lastly, remember to communicate your thought process and the value of your work clearly, as this is crucial for others to understand and trust your contributions.


In [42]:
# output = sentence_window_engine.query(
#     "How do you create your AI portfolio?")
# output.response

## Feedback functions

In [43]:
import nest_asyncio

nest_asyncio.apply()

Define Provider

In [44]:
from trulens.providers.openai import OpenAI
from trulens_eval.feedback.provider.endpoint.openai import OpenAIClient
from trulens_eval.utils.pyschema import Class
import openai as oai

# Define the client class and client kwargs
client_cls = Class.of_class(oai.OpenAI)
client_kwargs = {
    "api_key": "alanliuxiang",
    "base_url": "http://localhost:8989/v1"
}

# Initialize the OpenAIClient with the custom base URL
client = OpenAIClient(client_cls=client_cls, client_kwargs=client_kwargs)

provider = OpenAI(model_engine=MODEL_NAME,
                  client=client,
)

### 1. Answer Relevance

In [45]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [46]:
# from trulens_eval import TruLlama
from trulens.apps.llamaindex import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [47]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [48]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [49]:
import pandas as pd

data = {
    "query": ["what is AI?"],
    "query_id": ["1"],
    "expected_response": ["Artificial Intelligence"],
    "expected_chunks": [
        [
            {
                "text": "AI is the simulation of human intelligence processes by machines, especially computer systems.",
                "title": "AI is not a bubble :(",
                "expected_score": 0.9,
            },
            {
                "text": "AI is the evil overlod that's going to rule over all human beings.",
                "title": "AI should be feared",
                "expected_score": 0.4,
            },
            {
                "text": "AI is the future of humanity.",
                "title": "AI is the future",
                "expected_score": 0.5,
            },
        ],
    ],
}

df = pd.DataFrame(data)

In [50]:
session.add_ground_truth_to_dataset(
    dataset_name="test_dataset_ir",
    ground_truth_df=df,
    dataset_metadata={"domain": "Random IR dataset"},
)

In [51]:
ground_truth_df = session.get_ground_truth("test_dataset_ir")

In [52]:
ground_truth_df

,ground_truth_id,dataset_id,query,query_id,expected_response,expected_chunks,meta
0,ground_truth_hash_bf59adc430cc8f322ca9282cb76a4504,dataset_hash_b720d59fa18f93c3b8c936397adc99f7,what is AI?,1,Artificial Intelligence,"[{'text': 'AI is the simulation of human intelligence processes by machines, especially computer systems.', 'title': 'AI is not a bubble :(', 'expected_score': 0.9}, {'text': 'AI is the evil overlod that's going to rule over all human beings.', 'title': 'AI should be feared', 'expected_score': 0.4}, {'text': 'AI is the future of humanity.', 'title': 'AI is the future', 'expected_score': 0.5}]",{}


### 3. Groundedness

In [53]:
from trulens_eval.feedback import GroundTruthAgreement

grounded = GroundTruthAgreement(ground_truth_df,provider=provider)

In [54]:
f_groundedness = (
     # Feedback(grounded.groundedness_measure_with_cot_reasons,
    #          name="Groundedness"
    #         )
    Feedback(grounded.agreement_measure,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    # .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input prompt will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input response will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation of the RAG application

In [55]:
from trulens.apps.llamaindex import TruLlama

tru_recorder = TruLlama(
    # sentence_window_engine,
    query_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

In [56]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [57]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [58]:
eval_questions.append("How can I be successful in AI?")

In [59]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [60]:
for question in eval_questions:
    with tru_recorder as recording:
        query_engine.query(question)

In [61]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

app_name app_version                                     app_id  \
0    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
1    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
2    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
3    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
4    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 app_json  \
0  {'tru_class_info': {'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': [{'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': None}, {'name': 'App', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.app'}, 'bases': None}, {'name': 'AppDefinition', 'module': {'package_name': 'trulens.core.schema', 'module_name': 'trulens.core.schema.app'}, 'bases': None}, {'name': 'WithClassInfo', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.pyschema'}, 'bases': None}, {'name': 'SerialModel', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.serial'}, 'bases': None}, {'name': 'BaseModel', 'module': {'package_name': 'pydantic', 'module_name': 'pydantic.main'}, 'bases': None}, {'name': 'WithInstrumentCallbacks', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.instruments'}, 'bases': None}, {'name': 'Hashable', 'm

In [62]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output
0,How can I be successful in AI?,"To be successful in AI, focus on developing technical skills, particularly in areas like machine learning, data analysis, and programming. Engage in projects, both as part of your job and in your spare time, to gain practical experience and demonstrate your abilities. Participating in competitions like Kaggle can also be beneficial. Building a strong network through informational interviews and referrals can open doors to new opportunities. Collaboration and communication skills are crucial, as AI projects often require teamwork. Lastly, consider the importance of interpersonal and communication skills in working effectively in teams."
1,Is it normal to find parts of AI challenging?,"Yes, it is completely normal to find parts of AI challenging. The text suggests that many individuals, including those who have published seminal AI papers, have encountered similar technical difficulties. The author encourages those who struggle with the math behind linear regression, logistic regression, or neural networks to understand that everyone experiences such hurdles. It's important to remember that these challenges are part of the learning process in the AI field."
2,What are some common challenges in AI?,"Some common challenges in AI, as outlined in the context, include:\n\n1. Keeping up-to-date with rapidly evolving technology: The context emphasizes that AI is a nascent field with many technologies still in development, making it crucial to continuously learn and adapt.\n\n2. Iterative nature of AI projects: The high iteration required in AI projects can pose challenges in project management, as it's difficult to predict the timeline and return on investment, and further iterations may be necessary post-deployment to address drift.\n\n3. Collaborating with non-AI experts: Working on AI projects often involves collaborating with stakeholders who may lack AI expertise, which can present unique challenges.\n\n4. Overcoming imposter syndrome: Many individuals, including those who have published seminal AI papers, have faced technical challenges, indicating that imposter syndrome can be prevalent in the field.\n\n5. Choosing the right application area: While advancing basic AI technology is important, the context suggests that focusing on a specific application area can provide unique opportunities for creative work.\n\n6. Balancing depth and breadth of knowledge: The useful depth of knowledge required in AI can change over time, as techniques mature and become more reliable, potentially reducing the need for deep mathematical understanding.\n\n7. Team dynamics and communication: The context highlights the importance of having good teammates or people to discuss ideas with, indicating that effective collaboration and communication can be challenging.\n\n8. Determining appropriate metrics: AI teams often struggle with both machine learning and business metrics, requiring a balance between optimizing learning algorithms and addressing business-related concerns like user engagement and revenue."
3,What is the first step to becoming good at AI?,"The first step to becoming good at AI, as outlined in the context, is to ""suck at it."" This means starting with foundational skills and learning the basics, such as understanding models like linear regression, logistic regression, neural networks, decision trees, clustering, and anomaly detection. This initial phase of learning is crucial, as it sets the groundwork for more complex AI concepts."
4,Who are some accomplished individuals who have experienced imposter syndrome?,"Some accomplished individuals who have publicly spoken about experiencing imposter syndrome include Sheryl Sandberg, former Facebook COO; Michelle Obama, U.S. first lady; Tom Hanks, actor; and Mike Cannon-Brookes, co-CEO of Atlassian."
5,What is imposter syndrome and how does it relate to AI?,"Imposter syndrome is a psychological pattern where individuals, regardless of their achie

In [63]:
tru.get_leaderboard(app_ids=[])

,,latency,total_cost
app_name,app_version,,
App_1,base,2.172721,0.0


In [64]:
tru.run_dashboard()

Starting dashboard ...
Dashboard already running at path:   Local URL: http://localhost:48263



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>